# A Walk Through Forpus

[Forpus](https://severinsimmler.github.io/forpus) is a Python library for processing plain text corpora to various corpus formats. In most cases, each NLP tool uses its own idiosyncratic input format. This library helps you to convert a corpus very easy to the desired format.

To install Forpus, run the following command in your command-line:

```bash
pip install forpus
```

You will need a directory containing `.txt` files.

In [1]:
try:
    from forpus import forpus
except ModuleNotFoundError:
    import sys
    from pathlib import Path
    sys.path.insert(0, str(Path('.').absolute().parent))
    from forpus import forpus

## 1. Constructing the base class

In [12]:
SOURCE = Path('..', 'corpus')
FNAME_PATTERN = '{author}_{title}'

In [13]:
Corpus = forpus.Corpus(source=SOURCE,
                       target=Path('json'),
                       fname_pattern=FNAME_PATTERN)

In [14]:
Corpus.to_json()

In [18]:
with open(Path(TARGET, 'corpus.json'), 'r', encoding='utf-8') as file:
    print(json.load(file))

{'mary_doc3': {'author': 'mary', 'title': 'doc3', 'text': 'Mary has written the third and last document, but this is also pretty nice.\n'}, 'peter_doc1': {'author': 'peter', 'title': 'doc1', 'text': "This is the first document. It's written by Peter. And it contains a lot of words.\n"}, 'paul_doc2': {'author': 'paul', 'title': 'doc2', 'text': 'There is also a second document. This one is by Paul. Furthermore, this also contains a lot of tokens.\n'}}


In [17]:
corpus

{'mary_doc3': {'author': 'mary',
  'text': 'Mary has written the third and last document, but this is also pretty nice.\n',
  'title': 'doc3'},
 'paul_doc2': {'author': 'paul',
  'text': 'There is also a second document. This one is by Paul. Furthermore, this also contains a lot of tokens.\n',
  'title': 'doc2'},
 'peter_doc1': {'author': 'peter',
  'text': "This is the first document. It's written by Peter. And it contains a lot of words.\n",
  'title': 'doc1'}}